# Fuzzy Matching Company Names

In [1]:
import pandas as pd
import pymysql
from fuzzywuzzy import process
import db_info as db

companies = pd.read_csv('/Users/rachelwyatt/google (um)/MADS/capstone - siads 697-699/data/companies_sorted.csv')

connection = pymysql.connect(host=db.DB_HOST,
    user=db.DB_USER,
    password=db.DB_PASSWORD,
    database=db.DB_NAME,
    port=db.DB_PORT,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()
cursor.execute("SELECT * FROM jd;")
table = cursor.fetchall()
connection.close()

df = pd.json_normalize(table)

# (There are over 7 million company names in the company data file.)
# Filter companies to United States
companies = companies[companies.country=='united states']

# Create a unique list of all company names in the database
company_names = list(companies['name'][companies['name'].notna()].unique())

In [ ]:
# Perform fuzzy matching and get the top match from the company info for each company in the database
df['fuzzy_match_result'] = df['company_name'].apply(lambda x: process.extractOne(str(x), company_names))

# Extract top matched company and match score from the fuzzy match results
df['fuzzy_match_company'] = df['fuzzy_match_result'].apply(lambda x: x[0])
df['fuzzy_match_score'] = df['fuzzy_match_result'].apply(lambda x: x[1])

In [ ]:
df.to_csv('/Users/rachelwyatt/google (um)/MADS/capstone - siads 697-699/data/database_fuzzy_matched_US.csv')